In [58]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import statistics as st
import seaborn as sns
import datetime 
from geopy import distance
import folium
from folium.plugins import MarkerCluster
from folium.features import GeoJsonTooltip
from branca.colormap import LinearColormap
from collections import Counter
import geopy.distance


In [59]:
dc_capital_bike = pd.read_csv('mapping_dataset.csv')

In [60]:
print(dc_capital_bike[['start_lat', 'start_lng', 'end_lat', 'end_lng']].isnull().sum())


start_lat    0
start_lng    0
end_lat      0
end_lng      0
dtype: int64


In [61]:
df_spring_electric = dc_capital_bike[(dc_capital_bike['rideable_type'] == 'electric_bike') & (dc_capital_bike['Season'] == 'Spring')]

dc_capital_bike = df_spring_electric.dropna(subset=['start_lat', 'start_lng', 'end_lat', 'end_lng'])

dc_capital_bike[['start_lat', 'start_lng', 'end_lat', 'end_lng']] = dc_capital_bike[['start_lat', 'start_lng', 'end_lat', 'end_lng']].astype(float)


In [62]:
print(dc_capital_bike.dtypes)

ride_id                    float64
rideable_type               object
started_at                  object
ended_at                    object
start_station_name          object
start_station_id           float64
end_station_name            object
end_station_id             float64
start_lat                  float64
start_lng                  float64
end_lat                    float64
end_lng                    float64
member_casual               object
time_of_day                 object
duration_hours             float64
Date_year                    int64
Date_day                     int64
Date_dow_name               object
Date_month_name             object
Date_month_no                int64
day                          int64
dow                          int64
weekend                      int64
Season                      object
ride_count                   int64
hour                         int64
Minute                       int64
time                        object
distance_km         

In [63]:
dc_capital_bike['start_lat'] = pd.to_numeric(dc_capital_bike['start_lat'], errors='coerce')
dc_capital_bike['start_lng'] = pd.to_numeric(dc_capital_bike['start_lng'], errors='coerce')
dc_capital_bike['end_lat'] = pd.to_numeric(dc_capital_bike['end_lat'], errors='coerce')
dc_capital_bike['end_lng'] = pd.to_numeric(dc_capital_bike['end_lng'], errors='coerce')


In [64]:
# Drop rows where start or end station names are missing
dc_capital_bike = dc_capital_bike.dropna(subset=['start_station_name', 'end_station_name'])
dc_capital_bike['start_station_name'] = dc_capital_bike['start_station_name'].fillna("Unknown Station")
dc_capital_bike['end_station_name'] = dc_capital_bike['end_station_name'].fillna("Unknown Station")

dc_capital_bike['start_station_name'] = dc_capital_bike['start_station_name'].astype(str)
dc_capital_bike['end_station_name'] = dc_capital_bike['end_station_name'].astype(str)

# Count rides per start and end station
start_station_counts = dc_capital_bike['start_station_name'].value_counts().reset_index()
end_station_counts = dc_capital_bike['end_station_name'].value_counts().reset_index()

dc_capital_bike = dc_capital_bike.dropna(subset=['start_lat', 'start_lng', 'end_lat', 'end_lng'])


In [65]:
# import json
# with open (r'Capital_Bikeshare_locations.geojson') as i:
#     Capital_Bike_share  = json.loads(i.read())

# Capital_Bike_share = pd.DataFrame(Capital_Bike_share, columns=["name"])
# Capital_Bike_share["GDP"]=0

m3=folium.Map()

m3=folium.Map(location=[38.9072, -77.0369],   # Our map view centrepoint. In our case, Berlin.
                zoom_start=12,              # Starting zoom step
                max_zoom=26,                # Maximum zooming step
                min_zoom=2)                 # Minimum zooming step
m3

tools="Click here for more info!"

# folium.Marker([38.89894104, -77.078372598], popup='<a href="https://cathedral.org/">"Glover Park(P.S.E)").</a>',
#                 tooltip=tools).add_to(m3)

# folium.Marker([38.915544, -77.038252], popup='<a href="https://www.galatheatre.org/">GALA Hispanic Theatre.(P.S.C)").</a>',
#                 tooltip=tools).add_to(m3)
folium.Marker([38.887161, -77.037331], popup='<a href="https://www.nps.gov/nama/index.htm">National Mall").</a>',
                tooltip=tools).add_to(m3)
folium.Marker([38.9991, -77.0265], popup='<a href="https://www.traillink.com/trail/capital-crescent-trail/">"Crescent-trail").</a>',
                tooltip=tools).add_to(m3)
folium.Marker([38.3164, -77.0833], popup='<a href="https://www.mountvernon.org/">"Mount Vernon Estate.</a>',
                tooltip=tools).add_to(m3)

folium.Marker([38.890995, -77.2106], popup='Washington & Old Dominion Trail').add_to(m3)
                
folium.Marker([39.06703978, -77.16174354], popup='<a href=https://www.traillink.com/trail/carl-henn-millennium-trail/">"Carl Henn Millennium.</a>',
                tooltip=tools).add_to(m3)
folium.Marker([38.9586, -77.3570], popup='Downtown').add_to(m3)
              
folium.Marker([38.9687, -77.3411], popup='<a href=https://www.fxva.com/listing/lake-anne-plaza/2144/">"Lake Anne.</a>',
                tooltip=tools).add_to(m3)
folium.Marker([38.9878, -77.1003 ], popup='Southern Endpoint(Bethesda)').add_to(m3)
m3


In [66]:
# Filter for Spring and electric bikes
electric_spring = dc_capital_bike[(dc_capital_bike['Season'] == 'Spring') & (dc_capital_bike['rideable_type'] == 'electric_bike')]

# Convert 'started_at' to datetime format
dc_capital_bike['started_at'] = pd.to_datetime(dc_capital_bike['started_at'])

# Filter for Spring season (March, April, May)
electric_spring = dc_capital_bike[dc_capital_bike['started_at'].dt.month.isin([3, 4, 5])]

# Filter for electric bikes only
electric_bike_stations = electric_spring[electric_spring['rideable_type'] == 'electric_bike']

# Find the most used start stations (e.g., top 3)
top_stations = electric_bike_stations.nlargest(3, 'ride_count')

# Create a map centered around the top stations
m7 = folium.Map(location=[top_stations['start_lat'].mean(), top_stations['start_lng'].mean()], zoom_start=14)

# Add markers for most used start stations
for _, row in top_stations.iterrows():
    folium.Marker(
        location=[row['start_lat'], row['start_lng']],
        popup=f"{row['start_station_name']} - {row['ride_count']} rides",
        icon=folium.Icon(color="blue", icon="bicycle")
    ).add_to(m7)

# Display the map
m7


In [67]:
# Filter for electric bikes in Spring
df_spring_electric = dc_capital_bike[(dc_capital_bike['rideable_type'] == 'electric_bike') & (dc_capital_bike['Season'] == 'Spring')]

# Count rides per start station
station_counts = df_spring_electric['start_station_name'].value_counts().reset_index()
station_counts.columns = ['start_station_name', 'ride_count']

# Merge with station coordinates data
df_stations = dc_capital_bike[['start_station_name', 'start_lat', 'start_lng']].drop_duplicates()
station_counts = station_counts.merge(df_stations, on='start_station_name', how='left')

# Get the top 5 most used stations
most_used_stations = station_counts.nlargest(5, 'ride_count')

# Define the center of the map (e.g., mean latitude & longitude of top stations)
map_center = [most_used_stations['start_lat'].mean(), most_used_stations['start_lng'].mean()]

# Initialize the map
most_bike_map = folium.Map(location=map_center, zoom_start=12)

# Add marker clusters
marker_cluster = MarkerCluster().add_to(most_bike_map)

# Plot each most-used station on the map
for index, row in most_used_stations.iterrows():
    folium.Marker(
        location=[row['start_lat'], row['start_lng']],
        popup=f"Station: {row['start_station_name']}<br>Rides: {row['ride_count']}",
        icon=folium.Icon(color="green", icon="cloud")  # Green for most used
    ).add_to(marker_cluster)

# Show the map
most_bike_map



In [68]:
from folium.plugins import HeatMap
# Filter for electric bike rides in Spring
df_spring_electric = dc_capital_bike[(dc_capital_bike['rideable_type'] == 'electric_bike') & (dc_capital_bike['Season'] == 'Spring')]

# Extract relevant columns (latitude & longitude of start stations)
heatmap_data = df_spring_electric[['start_lat', 'start_lng']].dropna()

# Define the center of the map (average latitude & longitude)
map_center = [heatmap_data['start_lat'].mean(), heatmap_data['start_lng'].mean()]

# Initialize the map
most_bike_heatmap1 = folium.Map(location=map_center, zoom_start=12)

# Add the heatmap layer
HeatMap(heatmap_data.values, radius=10, blur=15, max_zoom=1).add_to(most_bike_heatmap1)

# Show the map
most_bike_heatmap1

In [69]:
# Filter only electric bike rides
dc_electric = dc_capital_bike[dc_capital_bike['rideable_type'] == 'electric_bike']

# Count the number of times each route (trail) is used
route_counts = dc_capital_bike.groupby(['start_station_name', 'end_station_name', 
                           'start_lat', 'start_lng', 
                           'end_lat', 'end_lng']).size().reset_index(name='ride_count')

# Get the most used trails (top 5)
top_routes = route_counts.nlargest(5, 'ride_count')

# Create a map centered around the average location of top trails
m9 = folium.Map(location=[top_routes['start_lat'].mean(), top_routes['start_lng'].mean()], zoom_start=13)

# Add markers for start stations
for _, row in top_routes.iterrows():
    folium.Marker(
        location=[row['start_lat'], row['start_lng']],
        popup=f"Start: {row['start_station_name']}",
        icon=folium.Icon(color="blue", icon="play")
    ).add_to(m9)

# Add markers for end stations
for _, row in top_routes.iterrows():
    folium.Marker(
        location=[row['end_lat'], row['end_lng']],
        popup=f"End: {row['end_station_name']}",
        icon=folium.Icon(color="red", icon="stop")
    ).add_to(m9)

# Add lines representing the most used trails
for _, row in top_routes.iterrows():
    folium.PolyLine(
        [(row['start_lat'], row['start_lng']), 
         (row['end_lat'], row['end_lng'])],
        color="green", weight=5, opacity=0.7, tooltip=f"Trail: {row['start_station_name']} → {row['end_station_name']} ({row['ride_count']} rides)"
    ).add_to(m9)

# Display the map
m9


In [70]:
Heatmap=folium.Map()

In [71]:
# Filter for Spring and electric bikes
electric_spring = dc_capital_bike[(dc_capital_bike['Season'] == 'Spring') & (dc_capital_bike['rideable_type'] == 'electric_bike')]

# Convert 'ended_at' to datetime format
dc_capital_bike['ended_at'] = pd.to_datetime(dc_capital_bike['ended_at'])

# Filter for Spring season (March, April, May)
electric_spring = dc_capital_bike[dc_capital_bike['ended_at'].dt.month.isin([3, 4, 5])]

# Filter for electric bikes only
electric_bike_stations = electric_spring[electric_spring['rideable_type'] == 'electric_bike']

# Find the most used start stations (e.g., top 3)
top_stations = electric_bike_stations.nlargest(3, 'ride_count')

# Create a map centered around the top stations
m11 = folium.Map(location=[top_stations['end_lat'].mean(), top_stations['end_lng'].mean()], zoom_start=14)

# Coordinates for the heatmap (end station locations in this case)
heatmap_data = []

# Add markers for most used end stations
for _, row in top_stations.iterrows():
    folium.Marker(
        location=[row['end_lat'], row['end_lng']],
        popup=f"{row['end_station_name']} - {row['ride_count']} rides",
        icon=folium.Icon(color="blue", icon="bicycle")
    ).add_to(m11)
    
    # Add end station coordinates to the heatmap data
    heatmap_data.append([row['end_lat'], row['end_lng']])

# Add the heatmap layer to the map
HeatMap(heatmap_data, radius=15, blur=25, max_zoom=1).add_to(m11)

# Display the map
m11

In [72]:
# Filter for electric bikes in Spring
df_spring_electric = dc_capital_bike[(dc_capital_bike['rideable_type'] == 'electric_bike') & (dc_capital_bike['Season'] == 'Spring')]

# Count rides per start station
station_counts = df_spring_electric['start_station_name'].value_counts().reset_index()
station_counts.columns = ['start_station_name', 'ride_count']

# Merge with station coordinates data
df_stations = dc_capital_bike[['start_station_name', 'start_lat', 'start_lng']].drop_duplicates()
station_counts = station_counts.merge(df_stations, on='start_station_name', how='left')

# Get the least used stations (e.g., bottom 5)
least_used_stations = station_counts.nsmallest(5, 'ride_count')

# Define the center of the map (e.g., mean latitude & longitude of stations)
map_center = [least_used_stations['start_lat'].mean(), least_used_stations['start_lng'].mean()]

# Initialize the map
least_bike_map = folium.Map(location=map_center, zoom_start=12)

# Add marker clusters
marker_cluster = MarkerCluster().add_to(least_bike_map)

# Plot each least-used station on the map
for index, row in least_used_stations.iterrows():
    folium.Marker(
        location=[row['start_lat'], row['start_lng']],
        popup=f"Station: {row['start_station_name']}<br>Rides: {row['ride_count']}",
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(marker_cluster)

# Show the map
least_bike_map


In [73]:
# Filter for electric bikes in Spring
df_spring_electric = dc_capital_bike[(dc_capital_bike['rideable_type'] == 'electric_bike') & (dc_capital_bike['Season'] == 'Spring')]

# Count rides per start station
station_counts = df_spring_electric['start_station_name'].value_counts().reset_index()
station_counts.columns = ['start_station_name', 'ride_count']

# Merge with station coordinates data
df_stations = dc_capital_bike[['start_station_name', 'start_lat', 'start_lng']].drop_duplicates()
station_counts = station_counts.merge(df_stations, on='start_station_name', how='left')

# Get the top 5 most used stations
most_used_stations = station_counts.nlargest(5, 'ride_count')

# Define the center of the map (e.g., mean latitude & longitude of top stations)
map_center = [most_used_stations['start_lat'].mean(), most_used_stations['start_lng'].mean()]

# Initialize the map
most_bike_map = folium.Map(location=map_center, zoom_start=12)

# Add marker clusters
marker_cluster = MarkerCluster().add_to(most_bike_map)

# Plot each most-used station on the map
for index, row in most_used_stations.iterrows():
    folium.Marker(
        location=[row['start_lat'], row['start_lng']],
        popup=f"Station: {row['start_station_name']}<br>Rides: {row['ride_count']}",
        icon=folium.Icon(color="green", icon="cloud")  # Green for most used
    ).add_to(marker_cluster)

# Show the map
most_bike_map



In [74]:
from folium.plugins import HeatMap
# Filter for electric bike rides in Spring
df_spring_electric = dc_capital_bike[(dc_capital_bike['rideable_type'] == 'electric_bike') & (dc_capital_bike['Season'] == 'Spring')]

# Extract relevant columns (latitude & longitude of start stations)
heatmap_data = df_spring_electric[['start_lat', 'start_lng']].dropna()

# Define the center of the map (average latitude & longitude)
map_center = [heatmap_data['start_lat'].mean(), heatmap_data['start_lng'].mean()]

# Initialize the map
most_bike_heatmap1 = folium.Map(location=map_center, zoom_start=12)

# Add the heatmap layer
HeatMap(heatmap_data.values, radius=10, blur=15, max_zoom=1).add_to(most_bike_heatmap1)

# Show the map
most_bike_heatmap1

In [75]:
# Filter for electric bikes in Spring
df_spring_electric = dc_capital_bike[(dc_capital_bike['rideable_type'] == 'electric_bike') & (dc_capital_bike['Season'] == 'Spring')]

# Count rides per end station
end_station_counts = df_spring_electric['end_station_name'].value_counts().reset_index()
end_station_counts.columns = ['end_station_name', 'ride_count']

# Merge with station coordinates data
df_stations = dc_capital_bike[['end_station_name', 'end_lat', 'end_lng']].drop_duplicates()
end_station_counts = end_station_counts.merge(df_stations, on='end_station_name', how='left')

# Get the top 5 most used end stations
most_used_end_stations = end_station_counts.nlargest(5, 'ride_count')

# Define the center of the map (e.g., mean latitude & longitude of top stations)
map_center = [most_used_end_stations['end_lat'].mean(), most_used_end_stations['end_lng'].mean()]

# Initialize the map
end_bike_map = folium.Map(location=map_center, zoom_start=12)

# Add marker clusters
marker_cluster = MarkerCluster().add_to(end_bike_map)

# Plot each most-used end station on the map
for index, row in most_used_end_stations.iterrows():
    folium.Marker(
        location=[row['end_lat'], row['end_lng']],
        popup=f"Station: {row['end_station_name']}<br>Rides: {row['ride_count']}",
        icon=folium.Icon(color="blue", icon="cloud")  # Blue for most used end stations
    ).add_to(marker_cluster)

# Show the map
end_bike_map


In [76]:
heatmap_data = heatmap_data.dropna()
heatmap_data_end = heatmap_data_end.dropna()
most_used_start_stations = most_used_start_stations.dropna(subset=['start_lat', 'start_lng'])
most_used_end_stations = most_used_end_stations.dropna(subset=['end_lat', 'end_lng'])

df_spring_electric[['start_lat', 'start_lng', 'end_lat', 'end_lng']] = df_spring_electric[['start_lat', 'start_lng', 'end_lat', 'end_lng']].apply(pd.to_numeric, errors='coerce')

if heatmap_data.empty or heatmap_data_end.empty:
    print("⚠️ No ride data found for electric bikes in Spring!")
else:
    HeatMap(heatmap_data.values, radius=10, blur=15).add_to(bike_map)
    HeatMap(heatmap_data_end.values, radius=10, blur=15, gradient={0.2: "blue", 0.5: "green", 0.8: "yellow", 1: "red"}).add_to(bike_map)


In [77]:
# Filter for electric bikes in Spring
df_spring_electric = dc_capital_bike[(dc_capital_bike['rideable_type'] == 'electric_bike') & (dc_capital_bike['Season'] == 'Spring')]

# Count rides per end station
end_station_counts = df_spring_electric['end_station_name'].value_counts().reset_index()
end_station_counts.columns = ['end_station_name', 'ride_count']

# Merge with station coordinates data
df_stations = dc_capital_bike[['end_station_name', 'end_lat', 'end_lng']].drop_duplicates()
end_station_counts = end_station_counts.merge(df_stations, on='end_station_name', how='left')

# Get the top 5 most used end stations
most_used_end_stations = end_station_counts.nlargest(5, 'ride_count')

# Define the center of the map (e.g., mean latitude & longitude of top stations)
map_center = [most_used_end_stations['end_lat'].mean(), most_used_end_stations['end_lng'].mean()]

# Initialize the map
end_bike_map = folium.Map(location=map_center, zoom_start=12)

# Add marker clusters
marker_cluster = MarkerCluster().add_to(end_bike_map)

# Plot each most-used end station on the map
for index, row in most_used_end_stations.iterrows():
    folium.Marker(
        location=[row['end_lat'], row['end_lng']],
        popup=f"Station: {row['end_station_name']}<br>Rides: {row['ride_count']}",
        icon=folium.Icon(color="blue", icon="cloud")  # Blue for most used end stations
    ).add_to(marker_cluster)

# Show the map
end_bike_map


In [78]:
# Filter for electric bike rides in Spring
df_spring_electric = dc_capital_bike[(dc_capital_bike['rideable_type'] == 'electric_bike') & (dc_capital_bike['Season'] == 'Spring')]

# Extract relevant columns (latitude & longitude of end stations)
heatmap_data = df_spring_electric[['end_lat', 'end_lng']].dropna()

# Define the center of the map (average latitude & longitude of end stations)
map_center = [heatmap_data['end_lat'].mean(), heatmap_data['end_lng'].mean()]

# Initialize the map
end_bike_heatmap1 = folium.Map(location=map_center, zoom_start=12)

# Add the heatmap layer
HeatMap(heatmap_data.values, radius=10, blur=15, max_zoom=1).add_to(end_bike_heatmap1)

# Show the map
end_bike_heatmap1

In [82]:
heatmap_data = heatmap_data.dropna()
heatmap_data_end = heatmap_data_end.dropna()
most_used_start_stations = most_used_start_stations.dropna(subset=['start_lat', 'start_lng'])
most_used_end_stations = most_used_end_stations.dropna(subset=['end_lat', 'end_lng'])

df_spring_electric[['start_lat', 'start_lng', 'end_lat', 'end_lng']] = df_spring_electric[['start_lat', 'start_lng', 'end_lat', 'end_lng']].apply(pd.to_numeric, errors='coerce')

if heatmap_data.empty or heatmap_data_end.empty:
    print("⚠️ No ride data found for electric bikes in Spring!")
else:
    HeatMap(heatmap_data.values, radius=10, blur=15).add_to(bike_map)
    HeatMap(heatmap_data_end.values, radius=10, blur=15, gradient={0.2: "blue", 0.5: "green", 0.8: "yellow", 1: "red"}).add_to(bike_map)


In [83]:
# Filter for electric bike rides in Spring
df_spring_electric = dc_capital_bike[(dc_capital_bike['rideable_type'] == 'electric_bike') & (dc_capital_bike['Season'] == 'Spring')]

# Extract relevant columns (latitude & longitude of start and end stations)
heatmap_data = df_spring_electric[['start_lat', 'start_lng']].dropna()
heatmap_data_end = df_spring_electric[['end_lat', 'end_lng']].dropna()

# Count rides per start station
start_station_counts = df_spring_electric['start_station_name'].value_counts().reset_index()
start_station_counts.columns = ['start_station_name', 'ride_count']

# Count rides per end station
end_station_counts = df_spring_electric['end_station_name'].value_counts().reset_index()
end_station_counts.columns = ['end_station_name', 'ride_count']

# Merge with station coordinates data
df_start_stations = dc_capital_bike[['start_station_name', 'start_lat', 'start_lng']].drop_duplicates()
df_end_stations = dc_capital_bike[['end_station_name', 'end_lat', 'end_lng']].drop_duplicates()

# Merge to get coordinates for most-used stations
start_station_counts = start_station_counts.merge(df_start_stations, on='start_station_name', how='left')
end_station_counts = end_station_counts.merge(df_end_stations, on='end_station_name', how='left')

# Get the top 5 most used start and end stations
most_used_start_stations = start_station_counts.nlargest(5, 'ride_count')
most_used_end_stations = end_station_counts.nlargest(5, 'ride_count')

# Define the map center (average latitude & longitude of start stations)
map_center = [heatmap_data['start_lat'].mean(), heatmap_data['start_lng'].mean()]

# Initialize the map
bike_map = folium.Map(location=map_center, zoom_start=12)

# Add the heatmap layer for start stations
HeatMap(heatmap_data.values, radius=10, blur=15, max_zoom=1).add_to(most_bike_heatmap1)

# Add the heatmap layer for end stations
HeatMap(heatmap_data_end.values, radius=10, blur=15, max_zoom=1, gradient={0.2: "blue", 0.5: "green", 0.8: "yellow", 1: "red"}).add_to(end_bike_heatmap1)

# Add marker clusters
marker_cluster = MarkerCluster().add_to(bike_map)

# Plot each most-used start station on the map (Green markers)
for index, row in most_used_start_stations.iterrows():
    folium.Marker(
        location=[row['start_lat'], row['start_lng']],
        popup=f"Start Station: {row['start_station_name']}<br>Rides: {row['ride_count']}",
        icon=folium.Icon(color="green", icon="cloud")
    ).add_to(marker_cluster)

# Plot each most-used end station on the map (Blue markers)
for index, row in most_used_end_stations.iterrows():
    folium.Marker(
        location=[row['end_lat'], row['end_lng']],
        popup=f"End Station: {row['end_station_name']}<br>Rides: {row['ride_count']}",
        icon=folium.Icon(color="blue", icon="cloud")
    ).add_to(marker_cluster)



In [84]:
# Filter for electric bike rides in Spring
df_spring_electric = dc_capital_bike[(dc_capital_bike['rideable_type'] == 'electric_bike') & (dc_capital_bike['Season'] == 'Spring')]

# Extract relevant columns (latitude & longitude of start and end stations)
heatmap_data = df_spring_electric[['start_lat', 'start_lng']].dropna()
heatmap_data_end = df_spring_electric[['end_lat', 'end_lng']].dropna()

# Count rides per start station
start_station_counts = df_spring_electric['start_station_name'].value_counts().reset_index()
start_station_counts.columns = ['start_station_name', 'ride_count']

# Count rides per end station
end_station_counts = df_spring_electric['end_station_name'].value_counts().reset_index()
end_station_counts.columns = ['end_station_name', 'ride_count']

# Merge with station coordinates data
df_start_stations = dc_capital_bike[['start_station_name', 'start_lat', 'start_lng']].drop_duplicates()
df_end_stations = dc_capital_bike[['end_station_name', 'end_lat', 'end_lng']].drop_duplicates()

# Merge to get coordinates for most-used stations
start_station_counts = start_station_counts.merge(df_start_stations, on='start_station_name', how='left')
end_station_counts = end_station_counts.merge(df_end_stations, on='end_station_name', how='left')

# Get the top 5 most used start and end stations
most_used_start_stations = start_station_counts.nlargest(5, 'ride_count')
most_used_end_stations = end_station_counts.nlargest(5, 'ride_count')

# Define the map center (average latitude & longitude of start stations)
map_center = [heatmap_data['start_lat'].mean(), heatmap_data['start_lng'].mean()]

# Initialize the map
bike_map = folium.Map(location=map_center, zoom_start=12)

# Add the heatmap layer for start stations
HeatMap(heatmap_data.values, radius=10, blur=15, max_zoom=1).add_to(most_bike_heatmap1)

# Add the heatmap layer for end stations
HeatMap(heatmap_data_end.values, radius=10, blur=15, max_zoom=1, gradient={0.2: "blue", 0.5: "green", 0.8: "yellow", 1: "red"}).add_to(end_bike_heatmap1)

# Add marker clusters
marker_cluster = MarkerCluster().add_to(bike_map)

# Plot each most-used start station on the map (Green markers)
for index, row in most_used_start_stations.iterrows():
    folium.Marker(
        location=[row['start_lat'], row['start_lng']],
        popup=f"Start Station: {row['start_station_name']}<br>Rides: {row['ride_count']}",
        icon=folium.Icon(color="green", icon="cloud")
    ).add_to(marker_cluster)

# Plot each most-used end station on the map (Blue markers)
for index, row in most_used_end_stations.iterrows():
    folium.Marker(
        location=[row['end_lat'], row['end_lng']],
        popup=f"End Station: {row['end_station_name']}<br>Rides: {row['ride_count']}",
        icon=folium.Icon(color="blue", icon="cloud")
    ).add_to(marker_cluster)

# Show the map
bike_map.save('Biky_map.html')

#bike_map.html

In [85]:
# Filter for electric bike rides in Spring
df_spring_electric = dc_capital_bike[(dc_capital_bike['rideable_type'] == 'electric_bike') & (dc_capital_bike['Season'] == 'Spring')]

# Extract relevant columns (latitude & longitude of start and end stations)
heatmap_data = df_spring_electric[['start_lat', 'start_lng']].dropna()
heatmap_data_end = df_spring_electric[['end_lat', 'end_lng']].dropna()

# Count rides per start station
start_station_counts = df_spring_electric['start_station_name'].value_counts().reset_index()
start_station_counts.columns = ['start_station_name', 'ride_count']

# Count rides per end station
end_station_counts = df_spring_electric['end_station_name'].value_counts().reset_index()
end_station_counts.columns = ['end_station_name', 'ride_count']

# Merge with station coordinates data
df_start_stations = dc_capital_bike[['start_station_name', 'start_lat', 'start_lng']].drop_duplicates()
df_end_stations = dc_capital_bike[['end_station_name', 'end_lat', 'end_lng']].drop_duplicates()

# Merge to get coordinates for most-used stations
start_station_counts = start_station_counts.merge(df_start_stations, on='start_station_name', how='left')
end_station_counts = end_station_counts.merge(df_end_stations, on='end_station_name', how='left')

# Get the top 5 most used start and end stations
most_used_start_stations = start_station_counts.nlargest(5, 'ride_count')
most_used_end_stations = end_station_counts.nlargest(5, 'ride_count')

# Define the map center (average latitude & longitude of start stations)
map_center = [heatmap_data['start_lat'].mean(), heatmap_data['start_lng'].mean()]

# Initialize the map
bike_map = folium.Map(location=map_center, zoom_start=12)

# Add the heatmap layer for start stations
HeatMap(heatmap_data.values, radius=10, blur=15, max_zoom=1).add_to(most_bike_heatmap1)

# Add the heatmap layer for end stations
HeatMap(heatmap_data_end.values, radius=10, blur=15, max_zoom=1, gradient={0.2: "blue", 0.5: "green", 0.8: "yellow", 1: "red"}).add_to(end_bike_heatmap1)

# Add marker clusters
marker_cluster = MarkerCluster().add_to(bike_map)

# Plot each most-used start station on the map (Green markers)
for index, row in most_used_start_stations.iterrows():
    folium.Marker(
        location=[row['start_lat'], row['start_lng']],
        popup=f"Start Station: {row['start_station_name']}<br>Rides: {row['ride_count']}",
        icon=folium.Icon(color="green", icon="cloud")
    ).add_to(marker_cluster)

# Plot each most-used end station on the map (Blue markers)
for index, row in most_used_end_stations.iterrows():
    folium.Marker(
        location=[row['end_lat'], row['end_lng']],
        popup=f"End Station: {row['end_station_name']}<br>Rides: {row['ride_count']}",
        icon=folium.Icon(color="blue", icon="cloud")
    ).add_to(marker_cluster)


#bike_map.html